# Correção de erros em informações da ccd

In [2]:
import os
import sys
import math
import locale
import pypdf
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from typing import List
from pydantic import BaseModel, Field
from utils_ccd import get_connection, get_info_file_path, extract_text_from_pdf, generate_pdf_office
from datetime import datetime
from dotenv import load_dotenv
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from pathlib import Path
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from docx2pdf import convert

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sql_informacoes = """ 
SELECT concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo, *
FROM processo.dbo.vw_ata_informacao ai 
WHERE ai.setor = 'CCD'
AND ai.Titulo_Modelo_informacao LIKE '%onitoramento%'
AND MONTH(ai.DataInclusao) IN (7,8,9)
"""
df_informacoes = pd.read_sql(sql_informacoes, get_connection())

In [4]:
df_informacoes['caminho_arquivo'] = df_informacoes.apply(get_info_file_path, axis=1)
df_informacoes['texto_pdf'] = df_informacoes['caminho_arquivo'].apply(extract_text_from_pdf)

In [28]:
df_fim = df_informacoes[~df_informacoes['texto_pdf'].str.contains('CGAD', na=False) & ~df_informacoes['texto_pdf'].str.contains('Cadastro', na=False)]
df_excluir = df_informacoes[df_informacoes['texto_pdf'].str.contains('CGAD', na=False) | df_informacoes['texto_pdf'].str.contains('Cadastro', na=False)]

In [29]:
len(df_fim), len(df_excluir)

(67, 41)

In [33]:
df_excluir['processo'] = df_excluir['numero_processo'].astype(str) + '/' + df_excluir['ano_processo'].astype(str)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_19536\3241232112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_excluir['processo'] = df_excluir['numero_processo'].astype(str) + '/' + df_excluir['ano_processo'].astype(str)


In [35]:
print(', '.join([f"'{proc}'" for proc in df_excluir['processo'].tolist()]))

'004352/2019', '006622/2015', '008670/2015', '004558/2024', '000050/2022', '007151/2015', '008348/2016', '003035/2022', '200238/2021', '004933/2020', '200064/2021', '014354/2014', '004214/2019', '200233/2021', '001881/2020', '004214/2019', '003404/2020', '002158/2019', '001678/2024', '000201/2017', '002146/2020', '004324/2017', '004715/2019', '008518/2018', '014254/2015', '015479/2017', '018129/2017', '100329/2020', '100366/2020', '100481/2020', '100498/2020', '100550/2020', '100817/2020', '101104/2022', '101411/2022', '102267/2019', '200088/2022', '200113/2023', '200164/2023', '200233/2021', '300039/2022'


In [31]:
for index, row in df_fim.iterrows():
    print("********************************")
    print(row['texto_pdf'])
    print("********************************")

********************************
 
______________________________________________________________________  
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Processual e Controle de Decisões 
Coordenadoria de Controle de Decisões 
 
 
Processo nº:  007099/2019 
Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
Interessado: PREFEITURA MUNICIPAL DE GALINHOS 
Relator: FRANCISCO POTIGUAR CAVALCANTI JÚNIOR 
 
INFORMAÇÃO INSTRUTIVA 
 
Trata-se de apuração de responsabilidade com fito de verificar o cumprimento 
das obrigações legais e normativas no tocante à divulgação de transparência da gestão 
fiscal da Prefeitura Municipal de Galinhos/RN.  
Superada a fase de instrução, o Acórdão nº 149/2020 – TC (Evento 37) julgou 
pela irregularidade das contas da Prefeitura Municipal de Galinh os nos termos do ar 75, 
inciso II, da Lei Complementar Estadual nº 464/2012, det

Retirar dos monitoramentos:

* Cadastro no CGAD

In [17]:
print(df_informacoes[['texto_pdf']].iloc[62].values[0])

 
______________________________________________________________________  
Av. Presidente Getúlio Vargas, 690 – Ed. Dr. Múcio Vilar Ribeiro Dantas 
CEP 59012-360 – Petrópolis, Natal/RN – www.tce.rn.gov.br  
1 
Diretoria de Instrução Processual e Controle de Decisões 
Coordenadoria de Controle de Decisões 
 
 
Processo nº:  001881/2020 
Assunto: APURAÇÃO DE RESPONSABILIDADE - PORTAL DA TRANSPARÊNCIA 2019. 
Interessado: CÂMARA MUNICIPAL DE SÃO MIGUEL DO GOSTOSO/RN 
Relator: ANA PAULA DE OLIVEIRA GOMES 
 
INFORMAÇÃO INSTRUTIVA 
 
Trata-se de apuração de responsabilidade com fito de verificar o cumprimento 
das obrigações legais e normativas no tocante à divulgação de transparência da gestão 
fiscal da Câmara Municipal de São Miguel do Gostoso/RN.  
Superada a fase de instrução, o Acórdão nº 355/2022 – TC (Evento 51) acolheu 
o voto da MM. Conselheira Relatora pela aprovação da matéria com ressalva, determinando 
que a gestão da função legislativa da Câmara Municipal de São Miguel do Gosto

In [11]:
len(df_informacoes)

108